<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [8]:
using System;
using System.Collections.Generic;
using System.Linq;

// Делегаты
public delegate void StudentEventHandler<T>(T student, EventArgs e) where T : Student;
public delegate void ScholarshipCalculationDelegate(Student student, ref double amount);
public delegate bool StudentFilterDelegate(Student student);

// События
public class StudentEventArgs : EventArgs
{
    public string Message { get; }
    public DateTime Timestamp { get; }

    public StudentEventArgs(string message)
    {
        Message = message;
        Timestamp = DateTime.Now;
    }
}

public class ScholarshipEventArgs : EventArgs
{
    public Student Student { get; }
    public double ScholarshipAmount { get; set; }

    public ScholarshipEventArgs(Student student, double amount)
    {
        Student = student;
        ScholarshipAmount = amount;
    }
}

public interface IPersonalInfo
{
    void DisplayBasicInfo();
    void UpdateContact(string email, string phone);
}

public interface IAcademic
{
    void DisplayAcademicDetails();
    void PerformStudy();
}

public interface IResearch
{
    void ConductResearch();
    void PublishPaper();
}

public interface IInternship
{
    void CompleteInternship();
    void DisplayInternshipInfo();
}

public class PersonalInfoService
{
    private readonly IPersonalInfo _person;

    public PersonalInfoService(IPersonalInfo person)
    {
        _person = person;
    }

    public void ProcessPersonalInfo()
    {
        Console.WriteLine("=== СЕРВИС ПЕРСОНАЛЬНОЙ ИНФОРМАЦИИ СТУДЕНТА ===");
        _person.DisplayBasicInfo();
    }

    public void UpdateContacts(string email, string phone)
    {
        _person.UpdateContact(email, phone);
    }
}

public class AcademicService
{
    private readonly IAcademic _student;

    public AcademicService(IAcademic student)
    {
        _student = student;
    }

    public void ProcessAcademicInfo()
    {
        Console.WriteLine("=== СЕРВИС АКАДЕМИЧЕСКОЙ ИНФОРМАЦИИ ===");
        _student.DisplayAcademicDetails();
        _student.PerformStudy();
    }
}

public class ScholarshipCalculator : IScholarshipCalculator
{
    // Событие для уведомления о расчете стипендии
    public event EventHandler<ScholarshipEventArgs> ScholarshipCalculated;

    // Коллекция дополнительных бонусов
    private readonly Dictionary<string, double> _specialBonuses = new Dictionary<string, double>
    {
        { "олимпиадник", 0.2 },
        { "активист", 0.1 },
        { "спортсмен", 0.15 }
    };

    // Делегат для пользовательских расчетов
    public ScholarshipCalculationDelegate CustomCalculation { get; set; }

    public void Calculate(Student student)
    {
        Calculate(student, 1000); // Базовая стипендия по умолчанию
    }

    public void Calculate(Student student, double baseAmount)
    {
        double scholarshipAmount = 0;
        
        if (student.AverageGrade >= 4.5)
        {
            scholarshipAmount = baseAmount * 1.5;
        }
        else if (student.AverageGrade >= 4.0)
        {
            scholarshipAmount = baseAmount;
        }

        // Применение пользовательского расчета через делегат
        if (CustomCalculation != null)
        {
            CustomCalculation(student, ref scholarshipAmount);
        }

        // Применение специальных бонусов
        scholarshipAmount = ApplySpecialBonuses(student, scholarshipAmount);

        // Вызов события
        ScholarshipCalculated?.Invoke(this, new ScholarshipEventArgs(student, scholarshipAmount));

        if (scholarshipAmount > 0)
        {
            Console.WriteLine($"Студент {student.Name} получает стипендию: {scholarshipAmount:F2} руб.");
        }
        else
        {
            Console.WriteLine($"Студент {student.Name} не получает стипендию");
        }
    }

    private double ApplySpecialBonuses(Student student, double baseAmount)
    {
        double bonusMultiplier = 1.0;

        // Проверка специальных бонусов
        if (student is BachelorStudent bachelor && !string.IsNullOrEmpty(bachelor.InternshipCompany))
        {
            bonusMultiplier += 0.1; // Бонус за стажировку
        }

        if (student is MasterStudent master && master.HasPublication)
        {
            bonusMultiplier += 0.15; // Бонус за публикации
        }

        if (student is PhDStudent phd && phd.PublicationsCount > 5)
        {
            bonusMultiplier += 0.2; // Бонус за много публикаций
        }

        return baseAmount * bonusMultiplier;
    }

    public void AddSpecialBonus(string bonusType, double multiplier)
    {
        _specialBonuses[bonusType] = multiplier;
    }
}

public interface IScholarshipCalculator
{
    void Calculate(Student student);
    void Calculate(Student student, double baseAmount);
    event EventHandler<ScholarshipEventArgs> ScholarshipCalculated;
}

public class DependencyContainer
{
    private readonly Dictionary<Type, object> _services = new Dictionary<Type, object>();

    public void Register<T>(T service)
    {
        _services[typeof(T)] = service;
    }

    public T Resolve<T>()
    {
        if (_services.TryGetValue(typeof(T), out var service))
        {
            return (T)service;
        }
        throw new InvalidOperationException($"Сервис {typeof(T)} не зарегистрирован");
    }
}

public class Student : IPersonalInfo, IAcademic
{
    // События
    public event EventHandler<StudentEventArgs> ContactInfoUpdated;
    public event EventHandler<StudentEventArgs> GradeUpdated;

    public string Name { get; set; }
    public int Age { get; set; }
    public int Course { get; set; }
    public string StudentId { get; set; }
    public double AverageGrade { get; set; }
    public string PhoneNumber { get; set; }
    public string Email { get; set; }
    public string Address { get; set; }
    public List<string> Subjects { get; set; }
    public DateTime EnrollmentDate { get; set; }

    // Коллекция для хранения истории изменений
    public List<string> ChangeHistory { get; private set; }

    protected readonly IScholarshipCalculator _scholarshipCalculator;

    public Student(string name, int age, int course, string studentId, double averageGrade, string phoneNumber)
        : this(name, age, course, studentId, averageGrade, phoneNumber, new ScholarshipCalculator())
    {
    }

    public Student(string name, int age, int course, string studentId, double averageGrade, 
                  string phoneNumber, IScholarshipCalculator scholarshipCalculator)
    {
        Name = name;
        Age = age;
        Course = course;
        StudentId = studentId;
        AverageGrade = averageGrade;
        PhoneNumber = phoneNumber;
        Subjects = new List<string>();
        ChangeHistory = new List<string>();
        _scholarshipCalculator = scholarshipCalculator;

        // Подписка на события калькулятора стипендий
        if (_scholarshipCalculator is ScholarshipCalculator calculator)
        {
            calculator.ScholarshipCalculated += OnScholarshipCalculated;
        }
    }

    private void OnScholarshipCalculated(object sender, ScholarshipEventArgs e)
    {
        if (e.Student == this)
        {
            AddToHistory($"Рассчитана стипендия: {e.ScholarshipAmount:F2} руб.");
        }
    }

    protected virtual void AddToHistory(string message)
    {
        ChangeHistory.Add($"{DateTime.Now:yyyy-MM-dd HH:mm:ss}: {message}");
    }

    public virtual void GetInfo()
    {
        Console.WriteLine($"Имя: {Name}, Возраст: {Age}, Курс: {Course}, ID: {StudentId}, Средний балл: {AverageGrade}, Телефон: {PhoneNumber}");
    }

    public virtual void Study()
    {
        Console.WriteLine($"Студент {Name} обучается в университете");
        AddToHistory("Зафиксирована учебная деятельность");
    }

    public virtual void TakeExam()
    {
        Console.WriteLine($"Студент {Name} успешно сдал экзамен");
        AddToHistory("Сдан экзамен");
    }

    public virtual void TakeExam(string subject)
    {
        Console.WriteLine($"Студент {Name} успешно сдал экзамен по предмету: {subject}");
        AddToHistory($"Сдан экзамен по {subject}");
    }

    public virtual void TakeExam(string subject, string professor)
    {
        Console.WriteLine($"Студент {Name} успешно сдал экзамен по предмету {subject} у профессора {professor}");
        AddToHistory($"Сдан экзамен по {subject} у {professor}");
    }

    public virtual void CalculateScholarship()
    {
        _scholarshipCalculator.Calculate(this);
    }

    public virtual void CalculateScholarship(double baseAmount)
    {
        _scholarshipCalculator.Calculate(this, baseAmount);
    }

    public virtual void DisplayContactInfo()
    {
        Console.WriteLine($"Контактная информация: Телефон: {PhoneNumber}" + 
                         (Email != null ? $", Email: {Email}" : "") + 
                         (Address != null ? $", Адрес: {Address}" : ""));
    }

    public void UpdateContactInfo(string newPhone)
    {
        PhoneNumber = newPhone;
        Console.WriteLine($"Номер телефона обновлен: {newPhone}");
        AddToHistory($"Обновлен телефон: {newPhone}");
        ContactInfoUpdated?.Invoke(this, new StudentEventArgs($"Обновлен телефон: {newPhone}"));
    }

    public void UpdateContactInfo(string newPhone, string newEmail)
    {
        PhoneNumber = newPhone;
        Email = newEmail;
        Console.WriteLine($"Контактная информация обновлена: Телефон - {newPhone}, Email - {newEmail}");
        AddToHistory($"Обновлены контакты: тел.{newPhone}, email.{newEmail}");
        ContactInfoUpdated?.Invoke(this, new StudentEventArgs("Обновлена контактная информация"));
    }

    public void UpdateContactInfo(string newPhone, string newEmail, string newAddress)
    {
        PhoneNumber = newPhone;
        Email = newEmail;
        Address = newAddress;
        Console.WriteLine($"Полная контактная информация обновлена");
        AddToHistory("Полностью обновлена контактная информация");
        ContactInfoUpdated?.Invoke(this, new StudentEventArgs("Полностью обновлена контактная информация"));
    }

    public virtual void DisplaySubjects()
    {
        if (Subjects != null && Subjects.Count > 0)
        {
            Console.Write($"Предметы студента {Name}: ");
            Console.WriteLine(string.Join(", ", Subjects));
        }
    }

    public virtual bool IsExcellentStudent()
    {
        return AverageGrade >= 4.8;
    }

    public void UpdateGrade(double newGrade)
    {
        double oldGrade = AverageGrade;
        AverageGrade = newGrade;
        AddToHistory($"Изменен средний балл: {oldGrade} -> {newGrade}");
        GradeUpdated?.Invoke(this, new StudentEventArgs($"Средний балл изменен на {newGrade}"));
    }

    public void DisplayChangeHistory()
    {
        Console.WriteLine($"\n=== История изменений студента {Name} ===");
        foreach (var entry in ChangeHistory)
        {
            Console.WriteLine(entry);
        }
    }

    void IPersonalInfo.DisplayBasicInfo()
    {
        Console.WriteLine($"Базовая информация студента: {Name}, {Age} лет, ID: {StudentId}, Курс: {Course}");
    }

    void IPersonalInfo.UpdateContact(string email, string phone)
    {
        this.Email = email;
        this.PhoneNumber = phone;
        Console.WriteLine($"Контакты студента обновлены: Email - {email}, Телефон - {phone}");
        AddToHistory($"Обновлены контакты через интерфейс");
    }

    void IAcademic.DisplayAcademicDetails()
    {
        Console.WriteLine($"Академические данные: Средний балл: {AverageGrade}, Курс: {Course}");
        Console.WriteLine($"Предметы: {(Subjects != null && Subjects.Count > 0 ? string.Join(", ", Subjects) : "не указаны")}");
    }

    void IAcademic.PerformStudy()
    {
        Study();
        Console.WriteLine($"Статус отличника: {(IsExcellentStudent() ? "Да" : "Нет")}");
    }
}

public class BachelorStudent : Student, IPersonalInfo, IAcademic, IInternship
{
    public string Specialty { get; set; }
    public string InternshipCompany { get; set; }
    public string GroupNumber { get; set; } 
    public int Semester { get; set; }
    public string Faculty { get; set; }
    public List<string> InternshipTasks { get; set; }
    public bool HasInternship { get; set; }

    // Коллекция для хранения достижений
    public List<string> Achievements { get; private set; }

    public event EventHandler<StudentEventArgs> InternshipCompleted;

    public BachelorStudent(string name, int age, int course, string specialty, 
                         string studentId, double averageGrade, string phoneNumber, string internshipCompany, 
                         string groupNumber, int semester, string faculty)
        : base(name, age, course, studentId, averageGrade, phoneNumber)
    {
        InitializeBachelorProperties(specialty, internshipCompany, groupNumber, semester, faculty);
    }

    public BachelorStudent(string name, int age, int course, string specialty, 
                         string studentId, double averageGrade, string phoneNumber, IScholarshipCalculator calculator,
                         string internshipCompany, string groupNumber, int semester, string faculty)
        : base(name, age, course, studentId, averageGrade, phoneNumber, calculator)
    {
        InitializeBachelorProperties(specialty, internshipCompany, groupNumber, semester, faculty);
    }

    private void InitializeBachelorProperties(string specialty, string internshipCompany, string groupNumber, int semester, string faculty)
    {
        Specialty = specialty;
        InternshipCompany = internshipCompany;
        GroupNumber = groupNumber;
        Semester = semester;
        Faculty = faculty;
        InternshipTasks = new List<string>();
        Achievements = new List<string>();
        HasInternship = !string.IsNullOrEmpty(internshipCompany);
    }

    public override void Study()
    {
        Console.WriteLine($"Студент {Name} обучается в университете, по специальности «{Specialty}»");
        AddToHistory($"Изучение специальности {Specialty}");
    }

    public override void GetInfo()
    {
        base.GetInfo();
        Console.WriteLine($"Специальность: {Specialty}, Стажировка: {InternshipCompany}, Группа: {GroupNumber}, Семестр: {Semester}, Факультет: {Faculty}");
    }

    public override void TakeExam(string subject)
    {
        Console.WriteLine($"Бакалавр {Name} сдает экзамен по специальному предмету: {subject} (специальность: {Specialty})");
        AddToHistory($"Сдан специальный экзамен по {subject}");
    }

    void IInternship.CompleteInternship()
    {
        Console.WriteLine($"Студент {Name} успешно завершил стажировку в компании {InternshipCompany}");
        AddToHistory($"Завершена стажировка в {InternshipCompany}");
        Achievements.Add($"Стажировка в {InternshipCompany}");
        InternshipCompleted?.Invoke(this, new StudentEventArgs($"Завершена стажировка в {InternshipCompany}"));
    }

    void IInternship.DisplayInternshipInfo()
    {
        Console.WriteLine($"Информация о стажировке: Компания: {InternshipCompany}, " +
                         $"Задачи: {(InternshipTasks.Count > 0 ? string.Join(", ", InternshipTasks) : "не назначены")}");
    }

    public void CompleteInternship()
    {
        ((IInternship)this).CompleteInternship();
    }

    public void CompleteInternship(string projectName)
    {
        Console.WriteLine($"Студент {Name} успешно завершил стажировку в компании {InternshipCompany} по проекту: {projectName}");
        AddToHistory($"Завершена стажировка по проекту {projectName}");
        Achievements.Add($"Проект: {projectName} в {InternshipCompany}");
        InternshipCompleted?.Invoke(this, new StudentEventArgs($"Завершена стажировка по проекту {projectName}"));
    }

    public void CompleteInternship(string projectName, string supervisor)
    {
        Console.WriteLine($"Студент {Name} успешно завершил стажировку в компании {InternshipCompany} " +
                         $"по проекту: {projectName} под руководством {supervisor}");
        AddToHistory($"Завершена стажировка под руководством {supervisor}");
        Achievements.Add($"Проект {projectName} под руководством {supervisor}");
        InternshipCompleted?.Invoke(this, new StudentEventArgs($"Завершена стажировка под руководством {supervisor}"));
    }

    public void DisplayInternshipInfo()
    {
        ((IInternship)this).DisplayInternshipInfo();
    }

    void IAcademic.DisplayAcademicDetails()
    {
        Console.WriteLine($"Академические данные бакалавра: {Specialty}, Факультет: {Faculty}");
        Console.WriteLine($"Группа: {GroupNumber}, Семестр: {Semester}, Средний балл: {AverageGrade}");
    }

    public override bool IsExcellentStudent()
    {
        return base.IsExcellentStudent() && HasInternship;
    }

    public void AddAchievement(string achievement)
    {
        Achievements.Add(achievement);
        AddToHistory($"Добавлено достижение: {achievement}");
    }

    public void DisplayAchievements()
    {
        Console.WriteLine($"\n=== Достижения студента {Name} ===");
        foreach (var achievement in Achievements)
        {
            Console.WriteLine($"- {achievement}");
        }
    }
}

public class MasterStudent : Student, IPersonalInfo, IAcademic, IResearch
{
    public string ScientificAdvisor { get; set; }
    public string ResearchTopic { get; set; }
    public string Laboratory { get; set; } 
    public string ResearchGrant { get; set; }
    public bool HasPublication { get; set; }
    public List<string> ResearchAreas { get; set; }
    public int ResearchProgress { get; set; }

    // Коллекции для исследований
    public List<string> Publications { get; private set; }
    public Dictionary<string, int> ResearchHours { get; private set; }

    // События
    public event EventHandler<StudentEventArgs> ResearchProgressChanged;
    public event EventHandler<StudentEventArgs> PaperPublished;

    public MasterStudent(string name, int age, int course, string scientificAdvisor,
                       string studentId, double averageGrade, string phoneNumber, string researchTopic, 
                       string laboratory, string researchGrant, bool hasPublication)
        : base(name, age, course, studentId, averageGrade, phoneNumber)
    {
        InitializeMasterProperties(scientificAdvisor, researchTopic, laboratory, researchGrant, hasPublication);
    }

    public MasterStudent(string name, int age, int course, string scientificAdvisor,
                       string studentId, double averageGrade, string phoneNumber, IScholarshipCalculator calculator,
                       string researchTopic, string laboratory, string researchGrant, bool hasPublication)
        : base(name, age, course, studentId, averageGrade, phoneNumber, calculator)
    {
        InitializeMasterProperties(scientificAdvisor, researchTopic, laboratory, researchGrant, hasPublication);
    }

    private void InitializeMasterProperties(string scientificAdvisor, string researchTopic, string laboratory, string researchGrant, bool hasPublication)
    {
        ScientificAdvisor = scientificAdvisor;
        ResearchTopic = researchTopic;
        Laboratory = laboratory;
        ResearchGrant = researchGrant;
        HasPublication = hasPublication;
        ResearchAreas = new List<string>();
        Publications = new List<string>();
        ResearchHours = new Dictionary<string, int>();
    }

    // Реализация метода ConductResearch из интерфейса IResearch
    void IResearch.ConductResearch()
    {
        WorkOnResearch();
    }

    void IResearch.PublishPaper()
    {
        HasPublication = true;
        string paperTitle = $"Исследование по теме: {ResearchTopic}";
        Publications.Add(paperTitle);
        Console.WriteLine($"Магистрант {Name} опубликовал научную статью: {paperTitle}");
        AddToHistory($"Опубликована статья: {paperTitle}");
        PaperPublished?.Invoke(this, new StudentEventArgs($"Опубликована статья: {paperTitle}"));
    }

    public override void TakeExam()
    {
        Console.WriteLine($"Студент {Name} успешно сдал экзамен. Экзамен принял преподаватель: {ScientificAdvisor}");
        AddToHistory($"Сдан экзамен у {ScientificAdvisor}");
    }

    public override void TakeExam(string subject)
    {
        Console.WriteLine($"Магистрант {Name} сдает исследовательский экзамен по предмету: {subject} " +
                         $"(научный руководитель: {ScientificAdvisor})");
        AddToHistory($"Сдан исследовательский экзамен по {subject}");
    }

    public override void GetInfo()
    {
        base.GetInfo();
        Console.WriteLine($"Научный руководитель: {ScientificAdvisor}, Тема исследования: {ResearchTopic}, Лаборатория: {Laboratory}, Грант: {ResearchGrant}, Публикации: {(HasPublication ? "Есть" : "Нет")}");
    }

    public void WorkOnResearch()
    {
        Console.WriteLine($"Магистрант {Name} работает над исследованием по теме: «{ResearchTopic}» в лаборатории {Laboratory}");
        AddToHistory($"Работа над исследованием: {ResearchTopic}");
    }

    public void WorkOnResearch(int hours)
    {
        Console.WriteLine($"Магистрант {Name} работает над исследованием «{ResearchTopic}» в течение {hours} часов");
        AddToHistory($"Работа над исследованием: {hours} часов");
    }

    public void WorkOnResearch(string methodology)
    {
        Console.WriteLine($"Магистрант {Name} проводит исследование по теме «{ResearchTopic}» " +
                         $"с использованием методики: {methodology}");
        AddToHistory($"Исследование с методикой: {methodology}");
    }

    public void DisplayResearchInfo()
    {
        Console.WriteLine($"Научная работа: Тема: {ResearchTopic}, Лаборатория: {Laboratory}, " +
                         $"Прогресс: {ResearchProgress}%, Области: {(ResearchAreas.Count > 0 ? string.Join(", ", ResearchAreas) : "не указаны")}");
    }

    void IAcademic.DisplayAcademicDetails()
    {
        Console.WriteLine($"Академические данные магистранта: Научный руководитель: {ScientificAdvisor}");
        Console.WriteLine($"Тема исследования: {ResearchTopic}, Лаборатория: {Laboratory}");
    }

    void IAcademic.PerformStudy()
    {
        WorkOnResearch();
        Console.WriteLine($"Наличие публикаций: {(HasPublication ? "Есть" : "Нет")}");
    }

    public void AddResearchArea(string area)
    {
        ResearchAreas.Add(area);
        AddToHistory($"Добавлена область исследований: {area}");
    }

    public void RecordResearchHours(string date, int hours)
    {
        ResearchHours[date] = hours;
        AddToHistory($"Записано {hours} часов исследований за {date}");
    }

    public void UpdateResearchProgress(int progress)
    {
        ResearchProgress = progress;
        AddToHistory($"Прогресс исследования: {progress}%");
        ResearchProgressChanged?.Invoke(this, new StudentEventArgs($"Прогресс исследования: {progress}%"));
    }

    public void DisplayResearchSummary()
    {
        Console.WriteLine($"\n=== Научная деятельность {Name} ===");
        Console.WriteLine($"Публикаций: {Publications.Count}");
        Console.WriteLine($"Области исследований: {string.Join(", ", ResearchAreas)}");
        Console.WriteLine($"Всего часов исследований: {ResearchHours.Values.Sum()}");
    }

    public override bool IsExcellentStudent()
    {
        return base.IsExcellentStudent() && HasPublication;
    }
}

public class PhDStudent : Student, IPersonalInfo, IAcademic, IResearch
{
    public string DissertationTopic { get; set; }
    public string ResearchField { get; set; }
    public int PublicationsCount { get; set; } 
    public string DefenseDate { get; set; }
    public string AcademicDegree { get; set; }
    public List<string> Conferences { get; set; }
    public string DissertationStatus { get; set; }

    // Коллекции для научной работы
    public List<string> PublishedPapers { get; private set; }
    public Dictionary<string, DateTime> ConferenceDates { get; private set; }

    // События
    public event EventHandler<StudentEventArgs> DefenseScheduled;
    public event EventHandler<StudentEventArgs> PaperAccepted;

    public PhDStudent(string name, int age, int course, string dissertationTopic,
                    string studentId, double averageGrade, string phoneNumber, string researchField, 
                    int publicationsCount, string defenseDate, string academicDegree)
        : base(name, age, course, studentId, averageGrade, phoneNumber)
    {
        InitializePhDProperties(dissertationTopic, researchField, publicationsCount, defenseDate, academicDegree);
    }

    public PhDStudent(string name, int age, int course, string dissertationTopic,
                    string studentId, double averageGrade, string phoneNumber, IScholarshipCalculator calculator,
                    string researchField, int publicationsCount, string defenseDate, string academicDegree)
        : base(name, age, course, studentId, averageGrade, phoneNumber, calculator)
    {
        InitializePhDProperties(dissertationTopic, researchField, publicationsCount, defenseDate, academicDegree);
    }

    private void InitializePhDProperties(string dissertationTopic, string researchField, int publicationsCount, string defenseDate, string academicDegree)
    {
        DissertationTopic = dissertationTopic;
        ResearchField = researchField;
        PublicationsCount = publicationsCount;
        DefenseDate = defenseDate;
        AcademicDegree = academicDegree;
        Conferences = new List<string>();
        PublishedPapers = new List<string>();
        ConferenceDates = new Dictionary<string, DateTime>();
        DissertationStatus = "В процессе";
    }

    // Реализация методов интерфейса IResearch
    void IResearch.ConductResearch()
    {
        Console.WriteLine($"Аспирант {Name} проводит исследование в области {ResearchField}");
        AddToHistory($"Проведено исследование в области {ResearchField}");
    }

    void IResearch.PublishPaper()
    {
        PublicationsCount++;
        string paperTitle = $"Диссертационное исследование: {DissertationTopic}";
        PublishedPapers.Add(paperTitle);
        Console.WriteLine($"Аспирант {Name} опубликовал научную статью. Общее количество публикаций: {PublicationsCount}");
        AddToHistory($"Опубликована статья. Всего публикаций: {PublicationsCount}");
        PaperAccepted?.Invoke(this, new StudentEventArgs($"Опубликована статья №{PublicationsCount}"));
    }

    public override void Study()
    {
        Console.WriteLine($"Аспирант {Name} пишет диссертацию по теме: «{DissertationTopic}»");
        AddToHistory($"Работа над диссертацией: {DissertationTopic}");
    }

    public override void GetInfo()
    {
        base.GetInfo();
        Console.WriteLine($"Тема диссертации: {DissertationTopic}, Область исследований: {ResearchField}, Публикаций: {PublicationsCount}, Защита: {DefenseDate}, Степень: {AcademicDegree}");
    }

    public void ConductResearch()
    {
        ((IResearch)this).ConductResearch();
    }

    public void ConductResearch(string method)
    {
        Console.WriteLine($"Аспирант {Name} проводит исследование в области {ResearchField} с использованием метода {method}");
        AddToHistory($"Исследование с методом: {method}");
    }

    public void PublishPaper()
    {
        ((IResearch)this).PublishPaper();
    }

    public void PublishPaper(string journal)
    {
        PublicationsCount++;
        PublishedPapers.Add($"Статья в {journal}: {DissertationTopic}");
        Console.WriteLine($"Аспирант {Name} опубликовал статью в журнале {journal}. Общее количество публикаций: {PublicationsCount}");
        AddToHistory($"Опубликована статья в {journal}");
        PaperAccepted?.Invoke(this, new StudentEventArgs($"Статья принята в {journal}"));
    }

    public void PrepareForDefense()
    {
        Console.WriteLine($"Аспирант {Name} готовится к защите диссертации {DefenseDate}. Публикаций: {PublicationsCount}");
        AddToHistory($"Подготовка к защите: {DefenseDate}");
    }

    public void PrepareForDefense(int monthsBefore)
    {
        Console.WriteLine($"Аспирант {Name} готовится к защите диссертации {DefenseDate} " +
                         $"(осталось {monthsBefore} месяцев). Публикаций: {PublicationsCount}");
        AddToHistory($"Подготовка к защите: осталось {monthsBefore} месяцев");
    }

    public void ScheduleDefense(string date)
    {
        DefenseDate = date;
        DissertationStatus = "Защита запланирована";
        Console.WriteLine($"Защита диссертации запланирована на {date}");
        AddToHistory($"Запланирована защита на {date}");
        DefenseScheduled?.Invoke(this, new StudentEventArgs($"Защита запланирована на {date}"));
    }

    public void AddConference(string conferenceName, DateTime date)
    {
        Conferences.Add(conferenceName);
        ConferenceDates[conferenceName] = date;
        AddToHistory($"Участие в конференции: {conferenceName}");
    }

    public void DisplayAcademicInfo()
    {
        Console.WriteLine($"Академическая информация: Статус диссертации: {DissertationStatus}, " +
                         $"Участие в конференциях: {(Conferences.Count > 0 ? string.Join(", ", Conferences) : "нет")}");
    }

    void IAcademic.DisplayAcademicDetails()
    {
        Console.WriteLine($"Академические данные аспиранта: Тема диссертации: {DissertationTopic}");
        Console.WriteLine($"Область исследований: {ResearchField}, Публикаций: {PublicationsCount}");
    }

    void IAcademic.PerformStudy()
    {
        ConductResearch();
        PrepareForDefense();
    }

    public void DisplayResearchSummary()
    {
        Console.WriteLine($"\n=== Научная деятельность аспиранта {Name} ===");
        Console.WriteLine($"Публикаций: {PublicationsCount}");
        Console.WriteLine($"Конференций: {Conferences.Count}");
        Console.WriteLine($"Статус диссертации: {DissertationStatus}");
    }

    public override bool IsExcellentStudent()
    {
        return base.IsExcellentStudent() && PublicationsCount >= 3;
    }
}

public class StudentManager<T> where T : Student
{
    private List<T> _students;
    private readonly IScholarshipCalculator _scholarshipCalculator;

    // События менеджера
    public event EventHandler<StudentEventArgs> StudentAdded;
    public event EventHandler<StudentEventArgs> StudentRemoved;

    // Делегаты для фильтрации
    public StudentFilterDelegate FilterDelegate { get; set; }

    public StudentManager(IScholarshipCalculator scholarshipCalculator)
    {
        _students = new List<T>();
        _scholarshipCalculator = scholarshipCalculator;
    }

    public int StudentCount => _students.Count;

    public void AddStudent(T student)
    {
        _students.Add(student);
        Console.WriteLine($"Добавлен студент: {student.Name}");
        
        // Подписка на события студента
        student.ContactInfoUpdated += OnStudentContactUpdated;
        student.GradeUpdated += OnStudentGradeUpdated;
        
        StudentAdded?.Invoke(this, new StudentEventArgs($"Добавлен студент: {student.Name}"));
    }

    public void RemoveStudent(T student)
    {
        if (_students.Remove(student))
        {
            Console.WriteLine($"Удален студент: {student.Name}");
            
            // Отписка от событий
            student.ContactInfoUpdated -= OnStudentContactUpdated;
            student.GradeUpdated -= OnStudentGradeUpdated;
            
            StudentRemoved?.Invoke(this, new StudentEventArgs($"Удален студент: {student.Name}"));
        }
    }

    private void OnStudentContactUpdated(object sender, StudentEventArgs e)
    {
        if (sender is T student)
        {
            Console.WriteLine($"Уведомление менеджера: контакты студента {student.Name} обновлены");
        }
    }

    private void OnStudentGradeUpdated(object sender, StudentEventArgs e)
    {
        if (sender is T student)
        {
            Console.WriteLine($"Уведомление менеджера: оценки студента {student.Name} обновлены");
        }
    }

    public T FindStudentById(string studentId)
    {
        return _students.Find(s => s.StudentId == studentId);
    }

    public void DisplayAllInfo()
    {
        Console.WriteLine($"\n=== Информация о всех студентах ({typeof(T).Name}) ===");
        foreach (var student in _students)
        {
            student.GetInfo();
            Console.WriteLine("---");
        }
    }

    public void DisplayAllContactInfo()
    {
        Console.WriteLine($"\n=== Контактная информация всех студентов ({typeof(T).Name}) ===");
        foreach (var student in _students)
        {
            student.DisplayContactInfo();
        }
    }

    public List<T> GetStudentsByCondition(Func<T, bool> condition)
    {
        return _students.FindAll(student => condition(student));
    }

    // Использование делегата для фильтрации
    public List<T> GetFilteredStudents()
    {
        if (FilterDelegate == null)
            return _students;

        return _students.Where(s => FilterDelegate(s)).ToList();
    }

    public void DisplayExcellentStudents()
    {
        var excellentStudents = GetStudentsByCondition(s => s.IsExcellentStudent());
        Console.WriteLine($"\n=== Отличники ({typeof(T).Name}) ===");
        foreach (var student in excellentStudents)
        {
            student.GetInfo();
        }
    }

    public void CalculateAllScholarships()
    {
        Console.WriteLine($"\n=== Стипендии всех студентов ({typeof(T).Name}) ===");
        foreach (var student in _students)
        {
            student.CalculateScholarship();
        }
    }

    // Работа с коллекциями через LINQ
    public void DisplayStudentsByGrade(double minGrade)
    {
        var filteredStudents = _students
            .Where(s => s.AverageGrade >= minGrade)
            .OrderByDescending(s => s.AverageGrade);

        Console.WriteLine($"\n=== Студенты со средним баллом >= {minGrade} ===");
        foreach (var student in filteredStudents)
        {
            Console.WriteLine($"{student.Name}: {student.AverageGrade}");
        }
    }

    public Dictionary<int, List<T>> GroupStudentsByCourse()
    {
        return _students
            .GroupBy(s => s.Course)
            .ToDictionary(g => g.Key, g => g.ToList());
    }

    public void DisplayCourseStatistics()
    {
        var courseGroups = GroupStudentsByCourse();
        
        Console.WriteLine("\n=== Статистика по курсам ===");
        foreach (var group in courseGroups)
        {
            Console.WriteLine($"Курс {group.Key}: {group.Value.Count} студентов, " +
                            $"средний балл: {group.Value.Average(s => s.AverageGrade):F2}");
        }
    }
}

// Демонстрация использования

var container = new DependencyContainer();
var calculator = new ScholarshipCalculator();
container.Register<IScholarshipCalculator>(calculator);

// Настройка пользовательского расчета через делегат
calculator.CustomCalculation = CustomScholarshipCalculation;

// Подписка на событие расчета стипендии
calculator.ScholarshipCalculated += OnScholarshipCalculated;

var scholarshipCalculator = container.Resolve<IScholarshipCalculator>();

Student myStudent = new Student("Виктор Анищенко", 19, 2, "ST001", 4.2, "+375 (29) 111-11-11", scholarshipCalculator);

BachelorStudent myBachelorStudent = new BachelorStudent("Александр Кислый", 20, 3, "Автоматизация технологических процессов", 
    "BS001", 4.7, "+375 (29) 123-45-67", scholarshipCalculator, "ТехноПром", "АТ-21-1", 5, "Факультет автоматизации");

// Подписка на события бакалавра
myBachelorStudent.InternshipCompleted += OnInternshipCompleted;

MasterStudent myMasterStudent = new MasterStudent("Олег Косой", 24, 1, "Анна Николаевна", "MS001", 4.9, 
    "+375 (29) 234-56-78", scholarshipCalculator, "Искусственный интеллект", "Лаборатория AI", "Грант №2024-AI-001", true);

PhDStudent myPhDStudent = new PhDStudent("Дмитрий Жидкий", 26, 1, "Исследование коррозионного разрушения", 
    "PHD001", 5.0, "+375 (29) 345-67-89", scholarshipCalculator, "Материаловедение", 8, "15.12.2025", "Кандидат технических наук");

// Подписка на события аспиранта
myPhDStudent.DefenseScheduled += OnDefenseScheduled;

// Создание менеджера с событиями
var bachelorManager = new StudentManager<BachelorStudent>(scholarshipCalculator);

// Подписка на события менеджера
bachelorManager.StudentAdded += OnStudentAdded;

// Настройка делегата фильтрации
bachelorManager.FilterDelegate = student => student.AverageGrade >= 4.5;

bachelorManager.AddStudent(myBachelorStudent);

var anotherBachelor = new BachelorStudent("Елена Светлова", 21, 4, "Программная инженерия", 
    "BS002", 4.9, "+375 (29) 456-78-90", scholarshipCalculator, "IT-Компания", 
    "ПИ-20-1", 7, "Факультет информатики");

bachelorManager.AddStudent(anotherBachelor);

// Демонстрация работы с коллекциями
Console.WriteLine("\n=== РАБОТА С КОЛЛЕКЦИЯМИ ===");
bachelorManager.DisplayCourseStatistics();
bachelorManager.DisplayStudentsByGrade(4.0);

// Демонстрация событий
Console.WriteLine("\n=== ДЕМОНСТРАЦИЯ СОБЫТИЙ ===");
myStudent.UpdateGrade(4.5);
myBachelorStudent.CompleteInternship("Автоматизация производства");
myBachelorStudent.AddAchievement("Победитель олимпиады по программированию");

// Демонстрация делегатов
Console.WriteLine("\n=== ДЕМОНСТРАЦИЯ ДЕЛЕГАТОВ ===");
var filteredStudents = bachelorManager.GetFilteredStudents();
Console.WriteLine("Отфильтрованные студенты (балл >= 4.5):");
foreach (var student in filteredStudents)
{
    Console.WriteLine($"- {student.Name}: {student.AverageGrade}");
}

// Демонстрация истории изменений
Console.WriteLine("\n=== ИСТОРИЯ ИЗМЕНЕНИЙ ===");
myBachelorStudent.DisplayChangeHistory();
myBachelorStudent.DisplayAchievements();

// Демонстрация работы с различными коллекциями
Console.WriteLine("\n=== РАБОТА С РАЗЛИЧНЫМИ КОЛЛЕКЦИЯМИ ===");

// Dictionary для хранения дополнительной информации
var studentAdditionalInfo = new Dictionary<string, string>
{
    { "BS001", "Стипендиат программы 'Будущие лидеры'" },
    { "BS002", "Участник научного сообщества" }
};

// HashSet для уникальных специальностей
var uniqueSpecialties = new HashSet<string>();
uniqueSpecialties.Add("Автоматизация технологических процессов");
uniqueSpecialties.Add("Программная инженерия");

Console.WriteLine("Уникальные специальности:");
foreach (var specialty in uniqueSpecialties)
{
    Console.WriteLine($"- {specialty}");
}

// Queue для обработки студентов
var studentQueue = new Queue<Student>();
studentQueue.Enqueue(myStudent);
studentQueue.Enqueue(myBachelorStudent);

Console.WriteLine("\nОчередь студентов на обработку:");
while (studentQueue.Count > 0)
{
    var student = studentQueue.Dequeue();
    Console.WriteLine($"Обработан: {student.Name}");
}

// Демонстрация интерфейсов
Console.WriteLine("\n=== ДЕМОНСТРАЦИЯ ИНТЕРФЕЙСОВ ===");
IResearch[] researchers = { myMasterStudent, myPhDStudent };
foreach (var researcher in researchers)
{
    researcher.ConductResearch();
    researcher.PublishPaper();
}


// Метод для делегата расчета стипендии
private static void CustomScholarshipCalculation(Student student, ref double amount)
{
// Дополнительная логика расчета
if (student.AverageGrade > 4.8)
{
    amount *= 1.1; // Дополнительный бонус для отличников
}

// Бонус за активность
if (student is BachelorStudent bachelor && bachelor.Achievements.Count > 2)
{
    amount *= 1.05;
}
}

// Обработчики событий
private static void OnScholarshipCalculated(object sender, ScholarshipEventArgs e)
{
Console.WriteLine($"РАССЧИТАНА СТИПЕНДИЯ: {e.Student.Name} - {e.ScholarshipAmount:F2} руб.");
}

private static void OnInternshipCompleted(object sender, StudentEventArgs e)
{
Console.WriteLine($"!!! ВАЖНО: {e.Message}");
}

private static void OnDefenseScheduled(object sender, StudentEventArgs e)
{
Console.WriteLine($" ЗАЩИТА: {e.Message}");
}

private static void OnStudentAdded(object sender, StudentEventArgs e)
{
Console.WriteLine($"СОБЫТИЕ МЕНЕДЖЕРА: {e.Message}");
}


Добавлен студент: Александр Кислый
СОБЫТИЕ МЕНЕДЖЕРА: Добавлен студент: Александр Кислый
Добавлен студент: Елена Светлова
СОБЫТИЕ МЕНЕДЖЕРА: Добавлен студент: Елена Светлова

=== РАБОТА С КОЛЛЕКЦИЯМИ ===

=== Статистика по курсам ===
Курс 3: 1 студентов, средний балл: 4.70
Курс 4: 1 студентов, средний балл: 4.90

=== Студенты со средним баллом >= 4 ===
Елена Светлова: 4.9
Александр Кислый: 4.7

=== ДЕМОНСТРАЦИЯ СОБЫТИЙ ===
Студент Александр Кислый успешно завершил стажировку в компании ТехноПром по проекту: Автоматизация производства
!!! ВАЖНО: Завершена стажировка по проекту Автоматизация производства

=== ДЕМОНСТРАЦИЯ ДЕЛЕГАТОВ ===
Отфильтрованные студенты (балл >= 4.5):
- Александр Кислый: 4.7
- Елена Светлова: 4.9

=== ИСТОРИЯ ИЗМЕНЕНИЙ ===

=== История изменений студента Александр Кислый ===
2025-11-30 20:41:18: Завершена стажировка по проекту Автоматизация производства
2025-11-30 20:41:18: Добавлено достижение: Победитель олимпиады по программированию

=== Достижения студента Але